# Welcome!!

This is another one of my from-scratch notebooks 🦴.  
I did one on Linear Regression a long time ago 📅🕐 and I highly recommend you check it out if you wanna see some basic concepts of Linear Regression!! 
With that said, I highly welcome any changes 😄 to the code to make it more modularized or efficient.  
### Feel absolutely free to write any changes in the comments for me 💭⌨. Alright let's jump in the code!!

# WARNING 🛑🛑🤚🤚💀- There is a part in this notebook where you are not supposed to put the output in the neural network. If you do, it will cause overflows in RAM. It is there only for educational purposes.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# The Basic Plan 🗺
### "A mind needs books like a sword needs a whetstone." ~ Tyrion Lannister 🧠📚🤺🥌

Hope ya like Game of Thrones, coz we will need the book. Not to sharpen our minds (they already are...) but to sharpen our model.  
We will train our Word2Vec model on Game of Thrones data. It will involve neural networks and a bit of preprocessing. I'll cover both CBOW as well as Skip-gram implementation in this.

### The basic war plan is this ⚔🛡:
1. Preprocessing: Preprocess the data (clean text if needed)
2. The Neural Network model: create, train, test
3. Create the Class: Make a custom Word2Vec class with the model
4. Visualization: Visualize the outputs

We'll do this for both the models CBOW as well as Skip-Gram. 

### One thing to consider is that there mainly 3️⃣ training methods for these 2 models:
1. Hierarchical softmax
2. Negative sampling
3. Naive softmax

We are naive little souls, so the 3rd one floats our boat the best.

# A. CBOW- Continuous Bag of Words

In [249]:
path = '/kaggle/input/game-of-thrones-books/' + '001ssb.txt'

In [250]:
txt = ""
with open(path, 'r', encoding='latin-1') as f:
    txt = f.read()

# 1. Preprocessing

### Preprocessing involves mainly the following steps:
- #### Cleaning the text
    - There's a whole lot of cleaning to be done in the text- pesky \n characters, single and double apostrophes, backslash characters etc.
- #### One Hot Encoding
    - This will form our sparse word vectors which will eventually form the input to the neural network model.
- #### k-Gram bag of words
    - This will be the various training tuples whose one hot encoded word vectors will be added and sent to the neural network model.

# 1a. Cleaning the text

In [251]:
import re

def get_clean_text(text):
    # Removes double apostrophes
    text = text.replace("''", "'")
    
    # Remove newline characters
    text = text.replace("\n", " ")
    
    # Remove backslashes
    text = text.replace("\\", "")
    
    # Remove any other special characters or punctuation
    text = re.sub(r'[^a-zA-Z\s\']', '', text)
    
    # Remove extra whitespace
    text = re.sub(' +', ' ', text)
    
    return text.lower()


In [252]:
clean_txt = get_clean_text(txt)

In [253]:
len(clean_txt)

1517298

In [254]:
len(clean_txt.split())

292919

In [255]:
clean_txt = ' '.join(clean_txt.split()[:int(1e5)])

#### Now we need to add 4 tokens- 2 start tokens and 2 end tokens to our text- this is for the context window and I'll explain this in a moment.
Let's assume the dollar symbol $ as our start token and the ampersand symbol & as the end token.

In [256]:
if 'STARTTOKEN' in clean_txt or 'ENDTOKEN' in clean_txt:
    print(True)
else:
    print(False)

False


#### So we can safely insert 'em

## Context window size

In [257]:
k = 2 # You can experiment with this if you want while running this notebook

In [258]:
start = ' '.join(['STARTTOKEN' for _ in range(k)])
end = ' '.join(['ENDTOKEN' for _ in range(k)])

In [259]:
start, end

('STARTTOKEN STARTTOKEN', 'ENDTOKEN ENDTOKEN')

In [260]:
semi_final_txt = start + ' ' + clean_txt + ' ' + end
semi_final_txt

"STARTTOKEN STARTTOKEN a game of thrones book one of a song of ice and fire by george r r martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hint of a smile gared did not rise to the bait he was an old man past fifty and he had seen the lordlings come and go dead is dead he said we have no business with the dead are they dead royce asked softly what proof have we will saw them gared said if he says they are dead that's proof enough for me will had known they would drag him into the quarrel sooner or later he wished it had been later rather than sooner my mother told me that dead men sing no songs he put in my wet nurse said the same thing will royce replied never believe anything you hear at a woman's tit there are things to be learned even from the dead his voice echoed too loud in the twilit forest page we have a long ride before us gared pointed out eight da

In [261]:
words = sorted(list(set(semi_final_txt.split())))

In [262]:
words

["'",
 "'arya",
 "'counting",
 "'have",
 "'it's",
 "'needle",
 "'s",
 "'shamed",
 "'the",
 "'thow",
 "'your",
 'ENDTOKEN',
 'STARTTOKEN',
 'a',
 'aback',
 'abandon',
 'abandoned',
 'abashed',
 'abed',
 'abide',
 'abilities',
 'able',
 'about',
 'above',
 'abreast',
 'abruptly',
 'absence',
 'absent',
 'absently',
 'absolution',
 'absorbed',
 'absurd',
 'absurdly',
 'abyss',
 'accent',
 'accentuated',
 'acceptance',
 'accepted',
 'accommodating',
 'accommodations',
 'accompany',
 'accord',
 'accosted',
 'account',
 'accurate',
 'accusation',
 'accusations',
 'accuse',
 'accustomed',
 'ache',
 'ached',
 'achievements',
 'aching',
 'acid',
 'acquiescence',
 'acre',
 'acres',
 'across',
 'act',
 'acted',
 'acting',
 'action',
 'activity',
 'actually',
 'added',
 'adders',
 'addition',
 'address',
 'adjusted',
 'admiration',
 'admiringly',
 'admit',
 'admitted',
 'admittedly',
 'admonition',
 'adorned',
 'adroitly',
 'adult',
 'advance',
 'advancing',
 'adventure',
 'adventures',
 'advice',

### Let's just remove the apstrophe words for simplicity's sake

In [263]:
idx = 0
for word in words:
    if word[0] != "'":
        break
    idx += 1
idx

11

In [264]:
words = words[idx:]

In [265]:
semi_final_txt = semi_final_txt.split()

In [266]:
final_txt = []
for word in semi_final_txt:
    if word[0] == "'":
        continue
    final_txt.append(word)

In [267]:
final_txt

['STARTTOKEN',
 'STARTTOKEN',
 'a',
 'game',
 'of',
 'thrones',
 'book',
 'one',
 'of',
 'a',
 'song',
 'of',
 'ice',
 'and',
 'fire',
 'by',
 'george',
 'r',
 'r',
 'martin',
 'prologue',
 'we',
 'should',
 'start',
 'back',
 'gared',
 'urged',
 'as',
 'the',
 'woods',
 'began',
 'to',
 'grow',
 'dark',
 'around',
 'them',
 'the',
 'wildlings',
 'are',
 'dead',
 'do',
 'the',
 'dead',
 'frighten',
 'you',
 'ser',
 'waymar',
 'royce',
 'asked',
 'with',
 'just',
 'the',
 'hint',
 'of',
 'a',
 'smile',
 'gared',
 'did',
 'not',
 'rise',
 'to',
 'the',
 'bait',
 'he',
 'was',
 'an',
 'old',
 'man',
 'past',
 'fifty',
 'and',
 'he',
 'had',
 'seen',
 'the',
 'lordlings',
 'come',
 'and',
 'go',
 'dead',
 'is',
 'dead',
 'he',
 'said',
 'we',
 'have',
 'no',
 'business',
 'with',
 'the',
 'dead',
 'are',
 'they',
 'dead',
 'royce',
 'asked',
 'softly',
 'what',
 'proof',
 'have',
 'we',
 'will',
 'saw',
 'them',
 'gared',
 'said',
 'if',
 'he',
 'says',
 'they',
 'are',
 'dead',
 "that's",

#### And there we go! Spick and span clean text and words🧼

# 1b. One Hot Encoding
### Now let's one hot encode the words quickly.

## QUICK NOTE- The method does not work if you have only 30GB of RAM (it failed for me too 😭). A better way is to use keras' own one_hot function.

Only see it for reference as a way to do it in bare bones one hot encoded fashion.

## 1b. i) The Garbage Way To Do It (Do not run the NN model via this method)

In [268]:
from sklearn.preprocessing import OneHotEncoder

In [269]:
ohe = OneHotEncoder()
ohe_words = ohe.fit_transform(np.array(words).reshape(-1, 1)).toarray()
ohe_words

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [270]:
ohe_words.shape

(7323, 7323)

We have nearly 7K words in our vocabulary as of now.  

#### IMP: Just for reference, the word vectors are always sorted in ascending order of ASCII values. Here's an example for reference:

In [271]:
a = ['b', 'a', 'c']

In [272]:
ohe = OneHotEncoder()
b = ohe.fit_transform(np.array(a).reshape(-1, 1)).toarray()
b

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

### Let's form the dictionary mapping of the words to their OHE word vectors.

In [273]:
ohe_wv = {word : wv for word, wv in zip(words, ohe_words)}

In [274]:
ohe_wv

{'ENDTOKEN': array([1., 0., 0., ..., 0., 0., 0.]),
 'STARTTOKEN': array([0., 1., 0., ..., 0., 0., 0.]),
 'a': array([0., 0., 1., ..., 0., 0., 0.]),
 'aback': array([0., 0., 0., ..., 0., 0., 0.]),
 'abandon': array([0., 0., 0., ..., 0., 0., 0.]),
 'abandoned': array([0., 0., 0., ..., 0., 0., 0.]),
 'abashed': array([0., 0., 0., ..., 0., 0., 0.]),
 'abed': array([0., 0., 0., ..., 0., 0., 0.]),
 'abide': array([0., 0., 0., ..., 0., 0., 0.]),
 'abilities': array([0., 0., 0., ..., 0., 0., 0.]),
 'able': array([0., 0., 0., ..., 0., 0., 0.]),
 'about': array([0., 0., 0., ..., 0., 0., 0.]),
 'above': array([0., 0., 0., ..., 0., 0., 0.]),
 'abreast': array([0., 0., 0., ..., 0., 0., 0.]),
 'abruptly': array([0., 0., 0., ..., 0., 0., 0.]),
 'absence': array([0., 0., 0., ..., 0., 0., 0.]),
 'absent': array([0., 0., 0., ..., 0., 0., 0.]),
 'absently': array([0., 0., 0., ..., 0., 0., 0.]),
 'absolution': array([0., 0., 0., ..., 0., 0., 0.]),
 'absorbed': array([0., 0., 0., ..., 0., 0., 0.]),
 'absur

### As is clear, it worked sweetly😀

# 1c. k-Gram Bag of Words

What does this mean? We will form a train-test tuple for our neural network. **k**, here, refers to the context window size. This means different things for different people, so let's clarify our own definition here:

A context window size of **k**, means that we will consider the previous and next k words as the input vectors to get our word vector for the middle target word. This is how CBOW gets the context words to predict a target word. 

In [275]:
input_words = []
targets = [] # if this goes well, our target_word list should contain exactly the same no. of words as the no. of words in the actual text

curcw = []

for i in range(len(final_txt) - 2 * k):
    if i == 0:
        curcw.extend([ final_txt[j] for j in range(i, i+ 2*k + 1) if j != i+k])
        input_words.append(curcw.copy())
        targets.append(final_txt[i+k])
        continue

    curcw.insert(k, final_txt[i+k-1]) 
    curcw.pop(k+1) 
    curcw = curcw[1:] 
    curcw.append(final_txt[i + 2 * k]) 
    
    input_words.append(curcw.copy())
    targets.append(final_txt[i+k])

In [276]:
for i in range(10):
    print('Context window:', input_words[i], '\t', 'Target word:', targets[i])

Context window: ['STARTTOKEN', 'STARTTOKEN', 'game', 'of'] 	 Target word: a
Context window: ['STARTTOKEN', 'a', 'of', 'thrones'] 	 Target word: game
Context window: ['a', 'game', 'thrones', 'book'] 	 Target word: of
Context window: ['game', 'of', 'book', 'one'] 	 Target word: thrones
Context window: ['of', 'thrones', 'one', 'of'] 	 Target word: book
Context window: ['thrones', 'book', 'of', 'a'] 	 Target word: one
Context window: ['book', 'one', 'a', 'song'] 	 Target word: of
Context window: ['one', 'of', 'song', 'of'] 	 Target word: a
Context window: ['of', 'a', 'of', 'ice'] 	 Target word: song
Context window: ['a', 'song', 'ice', 'and'] 	 Target word: of


### In case the logic is a bit confusing 😖, here's the explanation via a very simplified code with explanations in comments!
It's a bit more efficient than putting a double loop.

In [277]:
a = ['ok', 'this', 'is', 'awesome', 'very', 'nice', 'man', 'woman']

tmp_target = []
tmp_cw = []

xres = []

for i in range(len(a) - 2 * k):
    if i == 0:
        xres.extend([ a[j] for j in range(i, i+ 2*k + 1) if j != i+k])
        tmp_cw.append(xres.copy())
        tmp_target.append(a[i+k])
        continue

    xres.insert(k, a[i+k-1]) # add the previous target word back to context window at index just before the central word
    xres.pop(k+1) # remove the current target word
    xres = xres[1:] # pop the first element which was part of previous context window
    xres.append(a[i + 2 * k]) # add the latest element
    
    tmp_cw.append(xres.copy())
    tmp_target.append(a[i+k])
    
print(tmp_cw)
print(tmp_target)

del xres, tmp_cw, tmp_target

[['ok', 'this', 'awesome', 'very'], ['this', 'is', 'very', 'nice'], ['is', 'awesome', 'nice', 'man'], ['awesome', 'very', 'man', 'woman']]
['is', 'awesome', 'very', 'nice']


### But these aren't vectors. For that we will require our `ohe_wv` word vector dictionary to create the input and output tuples. So let's do that.

We will use the same logic as before, except we will append the word vectors this time instead of the word itself (the previous code was more for visualization).

In [278]:
X = []
y = []

cw = []
for i in range(len(final_txt) - 2 * k):
    if i == 0:
        cw.extend([ ohe_wv[final_txt[j]] for j in range(i, i+ 2*k + 1) if j != i+k])
        X.append(cw.copy())
        y.append(ohe_wv[final_txt[i+k]])
        continue

    cw.insert(k, ohe_wv[final_txt[i+k-1]]) 
    cw.pop(k+1) 
    cw = cw[1:] 
    cw.append(ohe_wv[final_txt[i + 2 * k]]) 
    
    X.append(cw.copy())
    y.append(ohe_wv[final_txt[i+k]])

In [279]:
for i in range(10):
    print('Context window:', X[i], '\t', 'Target word:', y[i])

Context window: [array([0., 1., 0., ..., 0., 0., 0.]), array([0., 1., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])] 	 Target word: [0. 0. 1. ... 0. 0. 0.]
Context window: [array([0., 1., 0., ..., 0., 0., 0.]), array([0., 0., 1., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])] 	 Target word: [0. 0. 0. ... 0. 0. 0.]
Context window: [array([0., 0., 1., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])] 	 Target word: [0. 0. 0. ... 0. 0. 0.]
Context window: [array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])] 	 Target word: [0. 0. 0. ... 0. 0. 0.]
Context window: [array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]

### ALL RIGHT IT WORKS!! 🥳🥳

## 1b. ii) The Better Way To Do It

In [280]:
!pip install keras_preprocessing

In [281]:
import keras_preprocessing
from keras_preprocessing.text import Tokenizer

In [394]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(final_txt)

In [395]:
sequences = tokenizer.texts_to_sequences(final_txt)
sequences

[[3051],
 [3051],
 [3],
 [946],
 [6],
 [1520],
 [782],
 [51],
 [6],
 [3],
 [1385],
 [6],
 [394],
 [2],
 [251],
 [76],
 [2024],
 [3052],
 [3052],
 [1521],
 [4220],
 [77],
 [192],
 [835],
 [61],
 [425],
 [1256],
 [18],
 [1],
 [623],
 [232],
 [4],
 [947],
 [239],
 [122],
 [37],
 [1],
 [2025],
 [47],
 [193],
 [66],
 [1],
 [193],
 [1257],
 [12],
 [65],
 [544],
 [499],
 [128],
 [19],
 [167],
 [1],
 [2026],
 [6],
 [3],
 [271],
 [425],
 [62],
 [21],
 [1085],
 [4],
 [1],
 [4221],
 [5],
 [8],
 [81],
 [94],
 [56],
 [324],
 [1753],
 [2],
 [5],
 [13],
 [220],
 [1],
 [4222],
 [111],
 [2],
 [113],
 [193],
 [25],
 [193],
 [5],
 [20],
 [77],
 [33],
 [29],
 [948],
 [19],
 [1],
 [193],
 [47],
 [27],
 [193],
 [499],
 [128],
 [624],
 [52],
 [1086],
 [33],
 [77],
 [38],
 [143],
 [37],
 [425],
 [20],
 [49],
 [5],
 [480],
 [27],
 [47],
 [193],
 [449],
 [1086],
 [202],
 [23],
 [45],
 [38],
 [13],
 [500],
 [27],
 [36],
 [4223],
 [15],
 [84],
 [1],
 [4224],
 [836],
 [83],
 [706],
 [5],
 [1155],
 [11],
 [13],
 [5

In [396]:
sequences[0], sequences[1], sequences[-1], sequences[-2]

([3051], [3051], [4219], [4219])

In [397]:
tokenizer.word_index # word index dictionary mapping

{'the': 1,
 'and': 2,
 'a': 3,
 'to': 4,
 'he': 5,
 'of': 6,
 'his': 7,
 'was': 8,
 'her': 9,
 'in': 10,
 'it': 11,
 'you': 12,
 'had': 13,
 'i': 14,
 'him': 15,
 'she': 16,
 'that': 17,
 'as': 18,
 'with': 19,
 'said': 20,
 'not': 21,
 'at': 22,
 'for': 23,
 'on': 24,
 'is': 25,
 'but': 26,
 'they': 27,
 'from': 28,
 'no': 29,
 'jon': 30,
 'were': 31,
 'my': 32,
 'have': 33,
 'all': 34,
 'be': 35,
 'would': 36,
 'them': 37,
 'will': 38,
 'ned': 39,
 'lord': 40,
 'when': 41,
 'your': 42,
 'there': 43,
 'so': 44,
 'me': 45,
 'up': 46,
 'are': 47,
 'bran': 48,
 'if': 49,
 'could': 50,
 'one': 51,
 'what': 52,
 'this': 53,
 'out': 54,
 'like': 55,
 'man': 56,
 'been': 57,
 'arya': 58,
 'their': 59,
 'eyes': 60,
 'back': 61,
 'did': 62,
 'looked': 63,
 'hand': 64,
 'ser': 65,
 'do': 66,
 'down': 67,
 'than': 68,
 'then': 69,
 'now': 70,
 'never': 71,
 'king': 72,
 'tyrion': 73,
 'page': 74,
 'boy': 75,
 'by': 76,
 'we': 77,
 'told': 78,
 'see': 79,
 'here': 80,
 'an': 81,
 'only': 82,
 'or

In [398]:
print(tokenizer.word_index['starttoken'],
tokenizer.word_index['endtoken'])

3051 4219


In [399]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 64
# k = 2 # window size

In [400]:
vocab_size

7324

In [401]:
X = []
y = []

cw = []
for i in range(len(sequences) - 2 * k):
    if i == 0:
        cw.extend([ sequences[j][0] for j in range(i, i+ 2*k + 1) if j != i+k])
        X.append(cw.copy())
        y.append(sequences[i+k][0])
        continue

    cw.insert(k, sequences[i+k-1][0]) 
    cw.pop(k+1) 
    cw = cw[1:] 
    cw.append(sequences[i + 2 * k][0]) 
    
    X.append(cw.copy())
    y.append(sequences[i+k][0])

In [402]:
for i in range(10):
    print('Context window:', X[i], '\t', 'Target word:', y[i])

Context window: [3051, 3051, 946, 6] 	 Target word: 3
Context window: [3051, 3, 6, 1520] 	 Target word: 946
Context window: [3, 946, 1520, 782] 	 Target word: 6
Context window: [946, 6, 782, 51] 	 Target word: 1520
Context window: [6, 1520, 51, 6] 	 Target word: 782
Context window: [1520, 782, 6, 3] 	 Target word: 51
Context window: [782, 51, 3, 1385] 	 Target word: 6
Context window: [51, 6, 1385, 6] 	 Target word: 3
Context window: [6, 3, 6, 394] 	 Target word: 1385
Context window: [3, 1385, 394, 2] 	 Target word: 6


In [403]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [404]:
import torch

In [405]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [406]:
X_train_t = torch.tensor(X_train).type(torch.LongTensor).to(device)
X_test_t = torch.tensor(X_test).type(torch.LongTensor).to(device)
y_train_t = torch.tensor(y_train).type(torch.LongTensor).to(device)
y_test_t = torch.tensor(y_test).type(torch.LongTensor).to(device)

In [407]:
X_train_t.shape, y_train_t.shape

(torch.Size([79990, 4]), torch.Size([79990]))

## Creating one hot vectors for our target values

In [408]:
y_train_t_ohe = torch.zeros(y_train_t.size(0), vocab_size).to(device)

for i, val in enumerate(y_train_t):
    idx = y_train_t[i].item()
    y_train_t_ohe[i][idx] = 1

In [409]:
y_test_t_ohe = torch.zeros(y_test_t.size(0), vocab_size).to(device)

for i, val in enumerate(y_test_t):
    idx = y_test_t[i].item()
    y_test_t_ohe[i][idx] = 1

In [410]:
X_train_t.device, X_test_t.device, y_train_t_ohe.device, y_test_t_ohe.device

(device(type='cuda', index=0),
 device(type='cuda', index=0),
 device(type='cuda', index=0),
 device(type='cuda', index=0))

# 2. The Neural Network model

We will use the true neural network architecture that is used in the actual CBOW model. We'll keep the no. of nodes in the hidden layer to be 100 for now. This will be the dense vector representation length of our word vectors in Word2Vec once the neural network is done learning.

### The plan is this:
- #### Take the input tuples and get a dense representation using the Embedding layer 🦷
    - This will condense our context window into one vector.
- #### Send the output from that layer to the Lambda layer
    - Get the average embeddings using a mean function.
- #### Send the output to a the output softmax layer 🧻
    - This will compute our logits for the final softmax function and compute the probability of each vector being the target word.
- #### Apply gradient descent via Adam optimizer and optimize the parameters in the nodes ⏳
    - This will make our model fit the data.
- #### Get the weights ⚖
    - At long last, we can get the model weights which are our word vector representation.

In [411]:
from torch import nn

In [412]:
class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        """
            Args:
                self (LambdaLayer) : the calling object
                lambd (function) : a lambda function that is to be implemented
        """
        super().__init__()
        self.lambd = lambd
        
    def forward(self, x):
        return self.lambd(x)

In [413]:
class CBOW(nn.Module):
    def __init__(self, lambd, vocab_size, embedding_dim=128):
        """
            Args:
                embedding_dim (int) : the embedding dimensions. This will be the final word2vec vector dimension
                lambd (function) : lambda function for word2vec, typically mean
        """
        super().__init__()
        
        self.embedding_dim = embedding_dim
        
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lambd = LambdaLayer(lambd)
        self.dense = nn.Linear(in_features=embedding_dim,
                              out_features=vocab_size)
        
    def forward(self, x):
        torch.manual_seed(42)
        torch.cuda.manual_seed(42)
        
        emb = self.embedding(x)
        mean = self.lambd(emb).to(device)
        logits = self.dense(mean)
        return logits

In [414]:
cbow = CBOW(lambd=(lambda x: torch.mean(x, axis=1)), vocab_size=vocab_size, embedding_dim=embedding_dim).to(device)

In [415]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=cbow.parameters(),
                            lr=0.03,
                           momentum=0.9)

In [416]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 512
traindata = DataLoader(TensorDataset(X_train_t, y_train_t_ohe), batch_size)

In [417]:
epochs = 200

for epoch in range(1, epochs+1):
    running_loss = 0
    
    for X_batch, y_batch in traindata:
        cbow.train()
        
        logits = cbow(X_batch)
        y_pred = torch.softmax(logits, dim=1)
        
        loss = loss_fn(logits, y_batch)
        running_loss += loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    cbow.eval()
    with torch.inference_mode():
        test_logits = cbow(X_test_t)
        test_preds = torch.softmax(test_logits, dim=1)
        
        test_loss = loss_fn(test_logits, y_test_t_ohe)
        
    if epoch % (epochs // 10) == 0:
        information = f"Epoch {epoch}\tTraining Loss: {running_loss / len(X_train_t)}\tTest Loss: {test_loss}"
        print(information)

Epoch 20	Training Loss: 0.012614648789167404	Test Loss: 6.552520751953125
Epoch 40	Training Loss: 0.011904866434633732	Test Loss: 6.323850631713867
Epoch 60	Training Loss: 0.011508170515298843	Test Loss: 6.232443332672119
Epoch 80	Training Loss: 0.011214476078748703	Test Loss: 6.18140983581543
Epoch 100	Training Loss: 0.010972060263156891	Test Loss: 6.149238109588623
Epoch 120	Training Loss: 0.010760792531073093	Test Loss: 6.128083229064941
Epoch 140	Training Loss: 0.010570960119366646	Test Loss: 6.114238262176514
Epoch 160	Training Loss: 0.010397166945040226	Test Loss: 6.105684280395508
Epoch 180	Training Loss: 0.0102360425516963	Test Loss: 6.101207733154297
Epoch 200	Training Loss: 0.010085338726639748	Test Loss: 6.100011825561523


In [418]:
cbow.state_dict()["embedding.weight"].shape

torch.Size([7324, 64])

## These are our vector embeddings!

In [420]:
emb = cbow.state_dict()["embedding.weight"].cpu().numpy()[1:] # don't need the first one, it's just paddings

In [421]:
start_idx, end_idx = tokenizer.word_index["starttoken"], tokenizer.word_index["endtoken"]
start_idx, end_idx

(3051, 4219)

In [422]:
emb.shape

(7323, 64)

In [423]:
cbow_emb = np.concatenate([emb[:start_idx-1], emb[start_idx + 1 - 1 : end_idx -1 ], emb[end_idx+1-1:]]) # don't need the token's emb either

In [424]:
cbow_emb.shape

(7321, 64)

# Visualizing the words

In [425]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(cbow_emb)

In [426]:
sim_matrix.shape

(7321, 7321)

In [427]:
del tokenizer.word_index["starttoken"], tokenizer.word_index["endtoken"]

In [428]:
int2word = {v : k for k, v in tokenizer.word_index.items()}

In [429]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'to': 4,
 'he': 5,
 'of': 6,
 'his': 7,
 'was': 8,
 'her': 9,
 'in': 10,
 'it': 11,
 'you': 12,
 'had': 13,
 'i': 14,
 'him': 15,
 'she': 16,
 'that': 17,
 'as': 18,
 'with': 19,
 'said': 20,
 'not': 21,
 'at': 22,
 'for': 23,
 'on': 24,
 'is': 25,
 'but': 26,
 'they': 27,
 'from': 28,
 'no': 29,
 'jon': 30,
 'were': 31,
 'my': 32,
 'have': 33,
 'all': 34,
 'be': 35,
 'would': 36,
 'them': 37,
 'will': 38,
 'ned': 39,
 'lord': 40,
 'when': 41,
 'your': 42,
 'there': 43,
 'so': 44,
 'me': 45,
 'up': 46,
 'are': 47,
 'bran': 48,
 'if': 49,
 'could': 50,
 'one': 51,
 'what': 52,
 'this': 53,
 'out': 54,
 'like': 55,
 'man': 56,
 'been': 57,
 'arya': 58,
 'their': 59,
 'eyes': 60,
 'back': 61,
 'did': 62,
 'looked': 63,
 'hand': 64,
 'ser': 65,
 'do': 66,
 'down': 67,
 'than': 68,
 'then': 69,
 'now': 70,
 'never': 71,
 'king': 72,
 'tyrion': 73,
 'page': 74,
 'boy': 75,
 'by': 76,
 'we': 77,
 'told': 78,
 'see': 79,
 'here': 80,
 'an': 81,
 'only': 82,
 'or

In [430]:
words = ["lord", "sister", "prince", "gods", "son", "girl"]

similar = []
for word in words:
    ohe_rep = tokenizer.word_index[word]
    sim_words = [int2word[idx] for idx in np.argsort(sim_matrix[ohe_rep])[:10]]
    similar.append(sim_words)
    
for word, sim_words in zip(words, similar):
    print(f"{word}: {sim_words}")

lord: ['recruit', 'pumpkins', 'bursting', 'translucent', 'crowds', 'murders', 'accosted', 'maybe', 'studying', 'honestly']
sister: ['deaf', 'beaten', 'moment', 'aghast', 'country', "he's", 'seen', 'flowered', 'refilling', 'serious']
prince: ['snorting', 'cost', 'huffing', 'opened', "day's", 'walk', 'earthen', 'glowered', 'swinging', 'peaceful']
gods: ['fortnight', 'spoils', 'crisp', 'mocking', 'net', 'redwyne', 'sign', 'rippling', 'curtain', 'desperate']
son: ['feathers', 'savaged', 'darted', 'stored', 'bouncing', 'hopes', 'wheels', 'morbid', 'useless', 'nephew']
girl: ["aegon's", 'exit', 'lyanna', 'baseborn', 'cradled', 'ghouls', 'insistent', 'poetry', 'ornate', 'hurting']


### Some of them make sense, like:
* "sign" for "gods"
* "lyanna" for girl
* "nephew" for son

# B. Skip-Gram

In [511]:
X_sg = []
y_sg = []

k = 10

# recall "k" is the context window size
for i in range(len(sequences) - 2 * k):
    for j in range(2 * k + 1):
        prob = np.random.randint(0, 2)
        rand_idx = np.random.randint(0, i) if i > len(sequences) // 2 else np.random.randint(i+2*k+1, len(sequences))
        
        if j != k:
            inject = [sequences[rand_idx][0], sequences[i+j][0]]
            X_sg.append([sequences[i+k][0], inject[prob]])
            y_sg.append(prob)

In [513]:
sequences[:20]

[[3051],
 [3051],
 [3],
 [946],
 [6],
 [1520],
 [782],
 [51],
 [6],
 [3],
 [1385],
 [6],
 [394],
 [2],
 [251],
 [76],
 [2024],
 [3052],
 [3052],
 [1521]]

In [512]:
for i in range(10):
    print('Context pair:', X_sg[i], '\t', 'Whether correct target:', y_sg[i])

Context pair: [1385, 3949] 	 Whether correct target: 0
Context pair: [1385, 3051] 	 Whether correct target: 1
Context pair: [1385, 1708] 	 Whether correct target: 0
Context pair: [1385, 946] 	 Whether correct target: 1
Context pair: [1385, 6] 	 Whether correct target: 1
Context pair: [1385, 1520] 	 Whether correct target: 1
Context pair: [1385, 782] 	 Whether correct target: 1
Context pair: [1385, 15] 	 Whether correct target: 0
Context pair: [1385, 6] 	 Whether correct target: 1
Context pair: [1385, 3] 	 Whether correct target: 1


In [514]:
from sklearn.model_selection import train_test_split
X_train_sg, X_test_sg, y_train_sg, y_test_sg = train_test_split(X_sg, y_sg, test_size=0.2, random_state=42)

In [577]:
X_train_t_sg = torch.tensor(X_train_sg).type(torch.LongTensor).to(device)
X_test_t_sg = torch.tensor(X_test_sg).type(torch.LongTensor).to(device)
y_train_t_sg = torch.tensor(y_train_sg).type(torch.float64).to(device)
y_test_t_sg = torch.tensor(y_test_sg).type(torch.float64).to(device)

In [578]:
X_train_t_sg.shape, y_train_t_sg.shape

(torch.Size([1599552, 2]), torch.Size([1599552]))

In [579]:
y_train_t_sg.dtype

torch.float64

In [544]:
a = torch.randn(512, 4)
b = torch.randn(512, 4)
# torch.tensordot(a, b, dims=[[1], [1]]).shape # dims=[[1], [0]] makes sure that the dimensions are matched together
torch.sum(a * b, dim=1, keepdim=True).shape

torch.Size([512, 1])

In [586]:
class Merge(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, vec_pair):
        vec1 = vec_pair[0].squeeze(dim=1)
        vec2 = vec_pair[1].squeeze(dim=1)
        return torch.sum(vec1 * vec2, dim=1, keepdim=True)

In [591]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64):
        """
            Args:
                embedding_dim (int) : the embedding dimensions. This will be the final word2vec vector dimension
                lambd (function) : lambda function for word2vec, typically mean
        """
        super().__init__()
        
        self.embedding_dim = embedding_dim
        
        self.embedding1 = nn.Embedding(num_embeddings=vocab_size, 
                                      embedding_dim=embedding_dim)
        self.embedding2 = nn.Embedding(num_embeddings=vocab_size, 
                                      embedding_dim=embedding_dim)
        self.merge = Merge()
        self.dense = nn.Linear(in_features=1,
                              out_features=1)
        
    def forward(self, x):
        torch.manual_seed(42)
        torch.cuda.manual_seed(42)
        
        emb1 = self.embedding1(x[:, 0])
        emb2 = self.embedding2(x[:, 1])
        
        dot_product = self.merge([emb1, emb2])
        
        logits = self.dense(dot_product)
        return logits
    
skip_gram = SkipGram(vocab_size).to(device)

In [592]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params=skip_gram.parameters(),
                           lr=0.001)

In [593]:
batch_size = 512
traindata_sg = DataLoader(TensorDataset(X_train_t_sg, y_train_t_sg), batch_size)

In [ ]:
epochs = 20
for epoch in range(1, epochs+1):
    running_loss = 0
    
    for X_batch, y_batch in traindata_sg:
        skip_gram.train()
        
        logits = skip_gram(X_batch).squeeze()
        y_pred = torch.round(torch.sigmoid(logits))
        
        loss = loss_fn(logits, y_batch)
        running_loss += loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        with torch.inference_mode():
            test_logits = skip_gram(X_test_t_sg).squeeze()
            test_preds = torch.round(torch.sigmoid(test_logits))

            test_loss = loss_fn(test_logits, y_test_t_sg)
        
    if epoch % (epochs // 10) == 0:
        information = f"Epoch {epoch}\tTraining Loss: {running_loss / len(X_train_t_sg)}\tTest Loss: {test_loss}"
        print(information)